#**Goals**
## In this section, we focused on developing regression and classification models to predict the urgency score of patient samples. Our first step involved data preparation to ensure the compatibility of the data with the models. Subsequently, we developed the models and then, proceeded to evaluate the performance of the models using various evaluation metrics, including accuracy and Mean Squared Error (MSE). Finally, we visualized the results through the use of different charts, providing a comprehensive understanding of the model's performance and predictive capabilities.

In [ ]:
!pip install pandas
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
from pylab import plot,show
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.model_selection import cross_val_score

In [ ]:
!pip install google.colab
from google.colab import drive
drive.mount('/content/d')

In [ ]:
# #Linoy's file path
file_path = '/content/d/MyDrive/final_project/data_for_regression_model.csv'
df_with_zeros = pd.read_csv(file_path)

# **Data Preperation**

In [ ]:
df_with_zeros

In [ ]:
df_with_zeros=df_with_zeros.drop(['Average_Urgency', 'Atlantic_Urgency', 'Unnamed: 0'], axis=1)

In [ ]:
df_with_zeros

## **Dealing with missing values**

In [ ]:
# Dropping zero values from urgency column
df = df_with_zeros.drop(df_with_zeros[df_with_zeros['Avera_Urgency'] == 0].index)
df

In [ ]:
# Save the DF to a CSV file
df.to_csv('df.csv', index=False)

In [ ]:
df['Avera_Urgency'].unique()

## **Dividing the target variable into bins**

In [ ]:
# Create the 'Urgency_Bin' column with custom bin labels
bin_labels = [1, 2, 3, 4, 5]  # Assign smaller numbers to bin 5 and larger numbers to bin 1
df['Urgency_Bin'] = pd.cut(df['Avera_Urgency'], bins=5, labels=bin_labels, right=False)

# Get the unique values in each bin
unique_values_by_bin = df.groupby('Urgency_Bin')['Avera_Urgency'].unique()

# Display the unique values in each bin
for bin_label, unique_values in unique_values_by_bin.items():
    print(f'Bin {bin_label}: {unique_values}')

In [ ]:
df['Urgency_Bin'].value_counts()

In [ ]:
df.shape

In [ ]:
df

In [ ]:
# Create a copy of the original DataFrame
df_binary = df.copy()

# Convert urgency values to binary categories
df_binary['Urgency_Bin'] = pd.cut(df_binary['Avera_Urgency'],2, labels=range(2))
df_binary['Urgency_Bin'].value_counts()

## **Removing columns**

In [ ]:
df=df.drop('Avera_Urgency', axis=1)
df_binary=df_binary.drop('Avera_Urgency', axis=1)

In [ ]:
df

In [ ]:
df_binary

In [ ]:
# Save the binary_df DataFrame to a CSV file
df_binary.to_csv('binary_df.csv', index=False)

In [ ]:
# Save the DF to a CSV file
df.to_csv('df.csv', index=False)

# **Using models to predict the urgency score**

## **Logistic Regression**

### **Grid Search for finding the best parameters for the model**

In [ ]:
# GridSearchCV is used to perform a grid search over the specified parameter grid.
# It exhaustively tries all the combinations of hyperparameters and evaluates each combination using cross-validation.
# The best combination of hyperparameters is determined based on the highest average score across the folds.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Select the relevant features and target
features = df.drop('Urgency_Bin', axis=1)
target = df['Urgency_Bin']

# Scale the features using StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Define the parameter grid for grid search
param_grid = {
    'penalty': ['l2', 'l1'],
    'C': [0.001, 0.1, 1.0, 10.0],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [100, 1000, 5000]
}

# Initialize the logistic regression model
model = LogisticRegression(multi_class='multinomial')

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(features_scaled, target)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)


### **The model**

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Select the relevant features and target
features = df.drop(['Urgency_Bin'], axis=1)
target = df['Urgency_Bin']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize the logistic regression model with L2 regularization
model = LogisticRegression(multi_class='multinomial', penalty='l1', solver='saga', max_iter=100, C=1.0)

# Fit the logistic regression model on the training features and target
model.fit(X_train, y_train)

# Get the feature names
feature_names = features.columns

# Print the coefficients (weights) of the logistic regression model
print("Coefficients:")
for feature, coefficient in zip(feature_names, model.coef_[0]):
    print(f"{feature} : {coefficient}")

# Predict urgency using the trained logistic regression model on the training and test features
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Create a DataFrame for the test results
test_df = X_test.copy()
test_df['Urgency_Bin'] = y_test
test_df['Predicted_Urgency'] = y_test_pred

# View the test DataFrame with predicted urgency
test_df

###**Evaluation metrics**

In [ ]:
#This code calculates the recall and precision scores for both the training and test sets based on the predicted values, from a logistic regression model.
# By setting average='macro', the scores are computed for each class independently and then averaged.

from sklearn.metrics import recall_score, precision_score

# Calculate recall on the training set
train_recall = recall_score(y_train, y_train_pred, average='macro')

# Calculate recall on the test set
test_recall = recall_score(y_test, y_test_pred, average='macro')

# Calculate precision on the training set
train_precision = precision_score(y_train, y_train_pred, average='macro')

# Calculate precision on the test set
test_precision = precision_score(y_test, y_test_pred, average='macro')

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print recall of the train and test
print("Train Recall:", train_recall)
print("Test Recall:", test_recall)


# Print precision of the train and test
print("Train Precision:", train_precision)
print("Test Precision:", test_precision)

# Print accuracy of the train and test
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

### **Visualizations**

In [ ]:
# The classification_report function is used to generate a comprehensive report of classification metrics, such as precision, recall, and F1-score,
# for a multi-class classification problem, with the specified target names and number of decimal digits.
from sklearn.metrics import classification_report
target_names = ['1', '2', '3', '4', '5']
print(classification_report(y_test, y_test_pred, target_names=target_names, digits=5))

In [ ]:
print(y_test.value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_test_pred, labels=[1, 2, 3, 4, 5])

# Set the figure size
plt.figure(figsize=(8, 6))

# Create a heatmap of the confusion matrix with smaller font size on the axis labels
heatmap = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"fontsize": 12}, xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])

# Set labels, title, and axis ticks
plt.xlabel("Predicted Class", fontsize=12)
plt.ylabel("True Class", fontsize=12)
plt.title("Confusion Matrix", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Set font size for the y-axis tick labels
heatmap.yaxis.set_tick_params(labelsize=10)

# Get the colorbar
colorbar = heatmap.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)

# Show the heatmap
plt.show()


In [ ]:
# Group the DataFrame by 'Priority' and 'Predicted_Urgency' and count the occurrences
grouped_data = test_df.groupby(['Priority', 'Predicted_Urgency']).size().unstack()

# Display the counts of each predicted urgency within each priority level
print(grouped_data)


In [ ]:
# Plot the distribution of Urgency_Bin
plt.hist(test_df['Urgency_Bin'], bins=5, alpha=0.5, label='Urgency_Bin')
# Plot the distribution of Predicted_Urgency
plt.hist(test_df['Predicted_Urgency'], bins=5, alpha=0.5, label='Predicted_Urgency')

plt.xlabel('Urgency Level', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Logistic regression- distribution of actual vs. predicted urgencies', fontsize=14)
plt.legend(fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()


In [ ]:
df.to_csv('logistic_regression_results.csv')

## **DecisionTree**

### **The model**

In [ ]:
# max_depth specifies the maximum depth of the decision tree. It controls the maximum number of levels in the tree.
# random_state sets the random seed for reproducibility. It ensures that the random number generator produces the same sequence of random numbers each time the code is run.
# param_grid is a dictionary that defines the grid of hyperparameters to search over, during the grid search. It specifies a range of values for max_depth from 1 to 10.
# GridSearchCV is a utility function for performing grid search with cross-validation.

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Select the relevant features and target
features = df.drop(['Urgency_Bin'], axis=1)
target = df['Urgency_Bin']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize the decision tree classifier with the best max_depth
model = DecisionTreeClassifier(max_depth=4, random_state=42)

# Train the model on the training set
model.fit(X_train, y_train)

# Predict the urgency on the test set
y_test_pred = model.predict(X_test)

# Predict urgency on the train set
y_train_pred = model.predict(X_train)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_test_pred)

print("Classification Report:\n", report)


### **Evaluation metrics**

In [ ]:
# Calculate recall on the training set
train_recall = recall_score(y_train, y_train_pred, average='macro')

# Calculate recall on the test set
test_recall = recall_score(y_test, y_test_pred, average='macro')

# Calculate precision on the training set
train_precision = precision_score(y_train, y_train_pred, average='macro')

# Calculate precision on the test set
test_precision = precision_score(y_test, y_test_pred, average='macro')

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print accuracy of the train and test
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

# Print recall of the train and test
print("Train Recall:", train_recall)
print("Test Recall:", test_recall)


# Print precision of the train and test
print("Train Precision:", train_precision)
print("Test Precision:", test_precision)

### **Visualizations**

In [ ]:
# Adjust the font size
plt.rcParams['font.size'] = 20

# Plot the decision tree
plt.figure(figsize=(10, 8))
plot_tree(model, feature_names=features.columns, class_names=model.classes_.astype(str),
          filled=True)

# Save the plot to a file
plt.savefig('decision_tree.png', dpi=300)

plt.show()

In [ ]:
print(y_test.value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_test_pred, labels=[1, 2, 3, 4, 5])

# Set the figure size
plt.figure(figsize=(8, 6))

# Create a heatmap of the confusion matrix with smaller font size on the axis labels
heatmap = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"fontsize": 12}, xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])

# Set labels, title, and axis ticks
plt.xlabel("Predicted Class", fontsize=12)
plt.ylabel("True Class", fontsize=12)
plt.title("Confusion Matrix", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Set font size for the y-axis tick labels
heatmap.yaxis.set_tick_params(labelsize=10)

# Get the colorbar
colorbar = heatmap.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)

# Show the heatmap
plt.show()


In [ ]:
df.to_csv('decision_tree_results.csv')

## **Linear Regression**

### **The model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Select the relevant features and target
features = df.drop(['Urgency_Bin'], axis=1)
target = df['Urgency_Bin']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Access the coefficients (weights) of the linear regression model
coefficients = model.coef_

# Print the coefficients
print("Coefficients:")
for feature, coefficient in zip(features.columns, coefficients):
    print(feature, ":", coefficient)

# Predict urgency using the linear regression model on the test set
y_test_pred = model.predict(X_test)

# Predict urgency on the train set
y_train_pred = model.predict(X_train)

# Round the predicted urgency values to the nearest integer and convert to int
y_test_pred_rounded = np.round(y_test_pred).astype(int)

# Add rounded predicted urgency values to the test DataFrame
test_df = X_test.copy()
test_df['Urgency_Bin'] = y_test
test_df['Predicted_Urgency'] = y_test_pred_rounded

# View the test DataFrame with predicted urgency
test_df

###**Evaluation metrics**

In [ ]:

# Calculate the mean squared error for training set
mse_train = mean_squared_error(y_train, y_train_pred)

# Calculate the mean absolute error for training set
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate the R-squared value for training set
r2_train = r2_score(y_train, y_train_pred)

# Calculate the mean squared error for test set
mse = mean_squared_error(y_test, y_test_pred_rounded)

# Calculate the mean absolute error for test set
mae = mean_absolute_error(y_test, y_test_pred_rounded)

# Calculate the R-squared value for test set
r2 = r2_score(y_test, y_test_pred_rounded)

# Print the R-squared value for training set
print("Train R-squared:", r2_train)

# Print the R-squared value for test set
print("Test R-squared:", r2)

# Print the mean squared error for training set
print("Train MSE:", mse_train)

# Print the mean squared error for test set
print("Test MSE:", mse)

# Print the mean absolute error for training set
print("Train MAE:", mae_train)

# Print the mean absolute error for test set
print("Test MAE:", mae)



### **Visualizations**

In [ ]:
# Plot the distribution of Urgency_Bin
plt.hist(test_df['Urgency_Bin'], bins=5, alpha=0.5, label='Urgency_Bin')
# Plot the distribution of Predicted_Urgency
plt.hist(test_df['Predicted_Urgency'], bins=5, alpha=0.5, label='Predicted_Urgency')

plt.xlabel('Urgency Level', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Linear regression- distribution of actual vs. predicted urgencies', fontsize=14)
plt.legend(fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()


In [ ]:
print(y_test.value_counts())

In [ ]:
# Plot the scatter plot
plt.scatter(y_test, y_test_pred_rounded)
plt.xlabel('Actual Urgency', fontsize=14)
plt.ylabel('Predicted Urgency', fontsize=14)
plt.title('Linear regression-  Actual vs. Predicted Urgency', fontsize=14)
plt.show()

In [ ]:
df.to_csv('linear_regression_results.csv')

##**Regression tree**

### **The model**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Select the relevant features and target
features = df.drop('Urgency_Bin', axis=1)
target = df['Urgency_Bin']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize the decision tree regressor
model = DecisionTreeRegressor()

# Train the model on the training set
model.fit(X_train, y_train)

# Predict urgency on the test set
y_test_pred = model.predict(X_test)

# Round the predicted urgency values to the nearest integer
y_pred_rounded = np.round(y_test_pred).astype(int)

# Predict urgency on the test set
y_train_pred = model.predict(X_train)

# Visualize the decision tree
plt.figure(figsize=(12, 8))
plot_tree(model, feature_names=features.columns, filled=True, rounded=True, precision=0)

# Save the plot to a file
plt.savefig('decision_tree.png', dpi=300)

plt.show()

### **Evaluation metrics**

In [ ]:
# Calculate the mean squared error for training set
mse_train = mean_squared_error(y_train, y_train_pred)

# Calculate the mean absolute error for training set
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate the R-squared value for training set
r2_train = r2_score(y_train, y_train_pred)

# Calculate the mean squared error for test set
mse = mean_squared_error(y_test, y_test_pred_rounded)

# Calculate the mean absolute error for test set
mae = mean_absolute_error(y_test, y_test_pred_rounded)

# Calculate the R-squared value for test set
r2 = r2_score(y_test, y_test_pred_rounded)

# Print the R-squared value for training set
print("Train R-squared:", r2_train)

# Print the R-squared value for test set
print("Test R-squared:", r2)

# Print the mean squared error for training set
print("Train MSE:", mse_train)

# Print the mean squared error for test set
print("Test MSE:", mse)

# Print the mean absolute error for training set
print("Train MAE:", mae_train)

# Print the mean absolute error for test set
print("Test MAE:", mae)

### **Visualizations**

In [ ]:
# Plot the distribution of Urgency_Bin in blue
plt.hist(y_test, bins=5, alpha=0.5, label='Urgency_Bin', density=True)

# Plot the distribution of Predicted_Urgency in orange
plt.hist(y_pred_rounded, bins=5, alpha=0.5, label='Predicted_Urgency', density=True)

plt.xlabel('Urgency Level', fontsize=14)
plt.ylabel('Density', fontsize=14)
plt.title('Regression Tree - Distribution of Actual vs. Predicted Urgencies', fontsize=14)
plt.legend(fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()


In [ ]:
# Scatter plot of actual vs. predicted urgencies
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred_rounded, alpha=0.5)
plt.xlabel('Actual Urgency')
plt.ylabel('Predicted Urgency')
plt.title('Regression Tree - Actual vs. Predicted Urgencies')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_test_pred, labels=[1, 2, 3, 4, 5])

# Set the figure size
plt.figure(figsize=(8, 6))

# Create a heatmap of the confusion matrix with smaller font size on the axis labels
heatmap = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"fontsize": 12}, xticklabels=[1, 2, 3, 4, 5], yticklabels=[1, 2, 3, 4, 5])

# Set labels, title, and axis ticks
plt.xlabel("Predicted Class", fontsize=12)
plt.ylabel("True Class", fontsize=12)
plt.title("Confusion Matrix", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Set font size for the y-axis tick labels
heatmap.yaxis.set_tick_params(labelsize=10)

# Get the colorbar
colorbar = heatmap.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)

# Show the heatmap
plt.show()

In [ ]:
# Group the DataFrame by 'Priority' and 'Predicted_Urgency' and count the occurrences
grouped_data = test_df.groupby(['Priority', 'Predicted_Urgency']).size().unstack()

# Display the counts of each predicted urgency within each priority level
print(grouped_data)

In [ ]:
df.to_csv('Regression_tree_results.csv')

##**DecisionTree with categorial output**

### **The model**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

# Select the relevant features and target
features = df_binary.drop('Urgency_Bin', axis=1)
target = df_binary['Urgency_Bin']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize the decision tree classifier
model = DecisionTreeClassifier()

# Train the model on the training set
model.fit(X_train, y_train)

# Predict the urgency on the test set
y_test_pred = model.predict(X_test)

# Predict urgency on the train set
y_train_pred = model.predict(X_train)


### **Evaluation metrics**

In [ ]:
# Calculate recall on the training set
train_recall = recall_score(y_train, y_train_pred, average='macro')

# Calculate recall on the test set
test_recall = recall_score(y_test, y_test_pred, average='macro')

# Calculate precision on the training set
train_precision = precision_score(y_train, y_train_pred, average='macro')

# Calculate precision on the test set
test_precision = precision_score(y_test, y_test_pred, average='macro')

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print accuracy of the train and test
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

# Print recall of the train and test
print("Train Recall:", train_recall)
print("Test Recall:", test_recall)


# Print precision of the train and test
print("Train Precision:", train_precision)
print("Test Precision:", test_precision)